In [ ]:
import os,sys
sys.path.append(r"../templates/")
import PYEO_model
sys.path.append(r"/opt/pyeo")
import pyeo.core as pyeo

from osgeo import gdal, gdal_array
import numpy as np

import geoTiff,dealwithShp
import pdb
import os
import s2_functions
import csv


In [ ]:
def s2_name(tiff_name):
    parts = tiff_name.split('_')
   # pdb.set_trace()
    date = parts[2][:8]; N = parts[3];row = parts[4];path = parts[5]
    return date, N, row, path

def match_mask(tiff):
    mask = tiff[:-4]+'.msk'
    return mask


In [ ]:
# Set input 
#images to process
s2_path = "  "

#shp of polygons where the NDVI will be generated and extracted
shp =  " .shp"

In [ ]:
# Set output 
# path to a table (.csv) where the NDVI number will be written to
out_csv = " "

In [ ]:
# caculate NDVI
tiff_list = s2_functions.search_files_fulldir(input_path=s2_path,search_type='end', search_key='.tif')
for tiff in tiff_list:
         print('------------------')
         print('working on: ' + tiff)
         g, s0 = s2_functions.readTiff(tiff,data_type=np.int32)
         NDVI = s2_functions.cal_vegIndex(s0=s0,s2res='10m')[:,:,0] 

         date, N, row, path = s2_name(os.path.basename(tiff))
         outname = os.path.join(os.path.dirname(tiff), date+ '_'+ N + '_' + row + '_' + path + '_NDVI.tif')
         print(outname)

         geoTiff.create(outname, g,s0,masked_NDVI,data_type=gdal.GDT_Int32)


In [ ]:
NDVI_list = s2_functions.search_files_fulldir(input_path=s2_path,search_type='end', search_key='_NDVI.tif')

f = open(out_csv,'w')
writer = csv.writer(f)

#
titles = ['date','id_1', 'id_2']
writer.writerow(titles)
#
first_row = True
#
for NDVI in NDVI_list:
         all = []
         date = os.path.basename(NDVI).split('_')[0]
         all.append(date)
         g = gdal.Open(NDVI)
         s0 = gdal_array.DatasetReadAsArray(g)
         regression, id, NDVI = dealwithShp_v3.getRaster(s0_fldr=s2_path, shpfile_path=shp, field_name='id', s0data=g,
                                                          nodata=0)
